# ILUSTRACION DE CREDIT SCORING

En este código se muestra cómo utilizar los métodos de regresión logística para la obtención de modelos de scoring

## 1. Objetivo

Utilizar las herramientas vistas en clase para obtener probabilidades en un modelo de scoring y contrastarlas con las probabilidades en el tiempo

## 2. Datos

Se utilizará la información de 60 trimestres de 5000 créditos hipotecarios de Estados Unidos tomada de protafolios bursatilizado (RMBSs) a partir del año 2000 (hasta 2015). El proveedor de la base es el International Financial Reasearch (www.internationalfinancialreaserch.org), y se tiene acceso a ella por la adquisición del libro Deep Credit Risk de Daniel Rosch y Harald Scheule, que es la principal fuente de este trabajo.

## 3. Desarrollo:

Importación de las librerías

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
import pandas as pd
import sklearn
import statsmodels.api as sm
import statsmodels.formula.api as smf
import cvxpy as cp

from numpy import linalg
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#import utils_pds as utils

import utils_pds2 as utils2

## Las siguientes librerias se usan en caso de correr en AWS
#import boto3 
#import io
#from io import StringIO

In [21]:
#Esta función se utiliza para determinar el número de dígitos de precisión que se imprimen
np.set_printoptions(precision=3, suppress=True)

In [22]:
# Estas líneas son para el caso de correr este código en AWS
### Conexión a la instancia, S3

##Definición de los parámetros para la conexión a S3
#s3client = boto3.client(
#    's3',
#    region_name='us-east-1'
#)

##Asignación del nombre del bucket de S3 y de los datos
#bucketname = "bucketmno2020arenitss"
#file_to_read = "data_t_final.csv"
#Definición objeto de S3 para conectarnos
#fileobj = s3client.get_object(
#    Bucket=bucketname,
#    Key=file_to_read
#    )

### Lectura del archivo, decodificación  y conversión a dataframe
#Lectura del archivo
#filedata = fileobj['Body'].read()

##Decodificación del archivo
#contenido = filedata.decode('utf-8')

##Tipo de archivo
#type(contenido)

##Conversión del archivo a dataframe
#StringData = StringIO(contenido)

##Lectura del archivo
#df = pd.read_csv(StringData)

In [23]:
df = pd.read_csv('/home/oscarperez/Documentos/OSCAR/MNO_2020/datos/data_t_final.csv')

In [24]:
df

,id,time,orig_time,first_time,mat_time,res_time,balance_time,LTV_time,interest_rate_time,rate_time,...,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,state_orig_time,hpi_orig_time,default_time,payoff_time,status_time,lgd_time,recovery_res
0,4,25,-2,25,119,NaN,61031.10,33.911009,10.500,5.90,...,587,81.8,10.5,KY,97.99,0,0,0,NaN,NaN
1,4,26,-2,25,119,NaN,60882.42,34.007232,10.500,5.90,...,587,81.8,10.5,KY,97.99,0,0,0,NaN,NaN
2,4,27,-2,25,119,NaN,60729.80,34.335349,10.500,5.90,...,587,81.8,10.5,KY,97.99,0,0,0,NaN,NaN
3,4,28,-2,25,119,NaN,60576.14,34.672545,10.875,5.90,...,587,81.8,10.5,KY,97.99,0,0,0,NaN,NaN
4,4,29,-2,25,119,NaN,60424.39,34.951639,10.875,5.90,...,587,81.8,10.5,KY,97.99,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62173,49972,52,25,52,145,NaN,180673.24,103.306966,7.500,5.15,...,708,79.8,7.5,AL,226.29,0,0,0,NaN,NaN
62174,49972,53,25,52,145,NaN,179944.95,95.736862,7.500,5.15,...,708,79.8,7.5,AL,226.29,0,0,0,NaN,NaN
62175,49972,54,25,52,145,NaN,179451.81,91.867079,7.500,5.15,...,708,79.8,7.5,AL,226.29,0,0,0,NaN,NaN
62176,49972,55,25,52,145,NaN,178952.48,91.560581,7.500,5.15,...,708,79.8,7.5,AL,226.29,0,0,0,NaN,NaN


Quitamos algunos nulos

In [25]:
data = df.dropna(subset=['time', 'default_time','LTV_time', 'FICO_orig_time','state_orig_time']).copy()

## 3.1 Armado de la base

A diferencia de un análisis de porbabilidades en el tiempo, un score se debe armar tomando en cuenta sólo datos de originación, a continuación se arma dicha base.

Primero nos quedamos sólo con datos de originaciómm

In [26]:
data2 = data[["id","REtype_CO_orig_time","REtype_PU_orig_time","REtype_SF_orig_time",
              "investor_orig_time",
              "balance_orig_time",
              "FICO_orig_time",
              "LTV_orig_time",
              "Interest_Rate_orig_time",
              "state_orig_time",
              "hpi_orig_time",
              "default_time"]]

Dejamos los creditos incumplidos

In [27]:
data_default = data2[(data2.default_time == 1)]

Ahora nos quedamos sólo con los créditos que no hicieron default

In [28]:
data_no_default_aux = data2[~data2.id.isin(data_default.id.values)]
data_no_default = data_no_default_aux.drop_duplicates(subset = ["id"])

Apendizamos la base. Con esta base se trabaja.

In [29]:
data_score = data_default.append(data_no_default)

In [30]:
data_score

,id,REtype_CO_orig_time,REtype_PU_orig_time,REtype_SF_orig_time,investor_orig_time,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,state_orig_time,hpi_orig_time,default_time
47,9,0,0,1,0,128000.0,501,80.0,8.000,GA,186.91,1
174,72,0,0,1,0,129500.0,536,69.6,7.605,FL,226.29,1
190,100,0,0,1,0,88000.0,556,80.0,7.810,OH,208.86,1
232,111,0,0,1,0,227000.0,622,64.9,7.325,NY,208.86,1
263,191,0,0,1,0,227200.0,630,80.0,7.360,FL,221.91,1
...,...,...,...,...,...,...,...,...,...,...,...,...
62128,49946,0,0,0,0,116450.0,555,85.0,2.000,PA,225.10,0
62137,49949,0,0,1,0,51300.0,619,90.0,5.500,VA,226.29,0
62146,49952,0,0,0,0,195500.0,537,85.0,1.998,AZ,225.10,0
62155,49956,0,0,0,0,81600.0,620,80.0,9.980,GA,225.10,0


Ahora obtenemos WOE e IV para hacer la discriminación de las variables con la función woe de utils y se agregan los valores de woe a nuestra base.

In [33]:
data_score, outputIV1 = utils2.woe(data_in = data_score, target='default_time', variable = 'balance_orig_time', bins=5, binning = 'True')
data_score, outputIV2 = utils2.woe(data_in = data_score, target='default_time', variable = 'FICO_orig_time', bins=5, binning = 'True')
data_score, outputIV3 = utils2.woe(data_in = data_score, target='default_time', variable = 'LTV_orig_time', bins=5, binning = 'True')
data_score, outputIV4 = utils2.woe(data_in = data_score, target='default_time', variable = 'Interest_Rate_orig_time', bins=5, binning = 'True')
data_score, outputIV5 = utils2.woe(data_in = data_score, target='default_time', variable = 'hpi_orig_time', bins=5, binning = 'True')
data_score, outputIV6 = utils2.woe(data_in = data_score, target='default_time', variable = 'REtype_CO_orig_time', bins=5, binning = 'False')
data_score, outputIV7 = utils2.woe(data_in = data_score, target='default_time', variable = 'REtype_PU_orig_time', bins=5, binning = 'False')
data_score, outputIV8 = utils2.woe(data_in = data_score, target='default_time', variable = 'REtype_SF_orig_time', bins=5, binning = 'False')


In [34]:
data_score

,id,REtype_CO_orig_time,REtype_PU_orig_time,REtype_SF_orig_time,investor_orig_time,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,state_orig_time,...,REtype_PU_orig_time_WOE_y,REtype_SF_orig_time_WOE_y,balance_orig_time_WOE,FICO_orig_time_WOE,LTV_orig_time_WOE,Interest_Rate_orig_time_WOE,hpi_orig_time_WOE,REtype_CO_orig_time_WOE,REtype_PU_orig_time_WOE,REtype_SF_orig_time_WOE
0,9,0,0,1,0,128000.0,501,80.0,8.000,GA,...,0.0,0.0,-0.098732,0.165367,0.144134,0.421176,-0.917694,0.0,0.0,0.0
1,2651,0,1,0,0,144400.0,591,80.0,8.890,CO,...,0.0,0.0,-0.098732,0.165367,0.144134,0.421176,-0.917694,0.0,0.0,0.0
2,26671,0,0,1,0,128000.0,565,80.0,9.300,IL,...,0.0,0.0,-0.098732,0.165367,0.144134,0.421176,-0.917694,0.0,0.0,0.0
3,5408,0,0,1,0,150000.0,520,75.0,8.700,FL,...,0.0,0.0,-0.098732,0.165367,0.144134,0.421176,-0.917694,0.0,0.0,0.0
4,12984,0,0,1,0,120000.0,499,80.0,8.875,MI,...,0.0,0.0,-0.098732,0.165367,0.144134,0.421176,-0.917694,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4974,37983,0,0,1,0,196000.0,694,70.0,7.000,FL,...,0.0,0.0,0.092890,-0.018758,-0.583179,0.415088,0.805075,0.0,0.0,0.0
4975,44886,0,0,0,0,178500.0,696,70.0,7.625,NY,...,0.0,0.0,0.092890,-0.018758,-0.583179,0.415088,0.805075,0.0,0.0,0.0
4976,33510,1,0,0,0,255000.0,702,65.6,7.125,MD,...,0.0,0.0,0.148418,-0.018758,-0.583179,0.415088,0.805075,0.0,0.0,0.0
4977,7025,0,0,1,1,400000.0,689,67.8,7.875,NC,...,0.0,0.0,-0.039459,-0.018758,-0.583179,0.415088,0.805075,0.0,0.0,0.0


Ordenamos por Information value para obtener las mejores variables

In [36]:
print(pd.concat([outputIV1, outputIV2, outputIV3, outputIV4, outputIV5, outputIV6, outputIV7, outputIV8]).round(decimals=4))

                      name      IV
0        balance_orig_time  0.0109
0           FICO_orig_time  0.0923
0            LTV_orig_time  0.0737
0  Interest_Rate_orig_time  0.1256
0            hpi_orig_time  0.5581
0      REtype_CO_orig_time  0.0000
0      REtype_PU_orig_time  0.0000
0      REtype_SF_orig_time  0.0000


Eliminamos las variables con IV menor a 0.02

In [37]:
# EN GENERAL SI EL IV
# < 0.02 muy bajo poder predcitivo
# > 0.02 y < 0.1 poder débil
# > 0.1 y < 0.3 poder moderado
# > 0.3 fuerte
# Excluimos las de bajo poder predictivo
data_score_regresion = data_score[["id","default_time", 
                         "FICO_orig_time_WOE", 
                         "LTV_orig_time_WOE", 
                         "Interest_Rate_orig_time_WOE", 
                         "hpi_orig_time_WOE"]]

Sobre estos valores se ajusta una regresión logistica con los métodos de clase programada en utils

In [38]:
utils2.estim_prob(data=data_score_regresion, variables = ['default_time', 'FICO_orig_time_WOE', 'LTV_orig_time_WOE', 'Interest_Rate_orig_time_WOE', 'hpi_orig_time_WOE'], default = "default_time")

4979
Evaluación de valor óptimo al resolver problema de optimización 5380.4697728304445


(array([0.227, 0.227, 0.227, ..., 0.4  , 0.4  , 0.4  ]),
 array([-0.822,  0.915,  0.938,  0.48 ,  0.973]))

Contrastamos los resultados con la función glm de python

In [39]:
model_score = smf.glm('default_time ~ FICO_orig_time_WOE + LTV_orig_time_WOE + Interest_Rate_orig_time_WOE + hpi_orig_time_WOE', family = sm.families.Binomial(), data = data_score_regresion).fit()
print(model_score.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:           default_time   No. Observations:                 4979
Model:                            GLM   Df Residuals:                     4974
Model Family:                Binomial   Df Model:                            4
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2690.2
Date:                Tue, 08 Dec 2020   Deviance:                       5380.5
Time:                        13:55:44   Pearson chi2:                 4.90e+03
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [40]:
# Se aplica el modelo a la base
PD_logit_model_score = pd.DataFrame(model_score.fittedvalues, columns=['PD_logit_model_score'])

In [41]:
# Hacemos una tabla con default o no default para cada credito que estamos estimando y su respectiva probabilidad
data2_score = pd.merge(data_score_regresion[['id', 'default_time']],PD_logit_model_score, right_index=True, left_index=True)

In [42]:
data2_score

,id,default_time,PD_logit_model_score
0,9,1,0.226808
1,2651,1,0.226808
2,26671,1,0.226808
3,5408,0,0.226808
4,12984,0,0.226808
...,...,...,...
4974,37983,1,0.400427
4975,44886,0,0.400427
4976,33510,0,0.400427
4977,7025,0,0.400427
